# Training pipeline

In [1]:
import os
import numpy as np
import scipy.io
from PIL import Image
import torch

if torch.backends.mps.is_available():
    print("MPS is available on your system!")
else:
    print("MPS is NOT available on your system.")

MPS is available on your system!


In [2]:
BASE_OUTPUT_DATA_DIR = 'rodents'
YAML_FILE = 'rodent.yaml'

In [3]:
RODENT_YAML_CONENT = """
# CAR

# download command/URL (optional)
download: echo "no download"

# train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: ../../data/dataset/train/images
val: ../../data/dataset/val/images
test:  ../../data/dataset/test/images

# number of classes
nc: 1

# class names
names: ["rodents"]
"""

with open(YAML_FILE, 'wt') as file:
    file.write(RODENT_YAML_CONENT) 

# Run the training

In [4]:
!python train.py \
    --workers 8 \
    --device 0 \
    --batch-size 2 \
    --data rodent.yaml \
    --img 512 512 \
    --cfg cfg/training/yolov7-tiny.yaml \
    --weights yolov7-tiny.pt \
    --name yolov7-car-detector \
    --hyp data/hyp.scratch.custom.yaml \
    --epochs 15 

YOLOR 🚀 88eadf0 torch 2.2.0 MPS

Namespace(weights='yolov7-tiny.pt', cfg='cfg/training/yolov7-tiny.yaml', data='rodent.yaml', hyp='data/hyp.scratch.custom.yaml', epochs=15, batch_size=2, img_size=[512, 512], rect=False, resume=False, nosave=False, notest=False, noautoanchor=False, evolve=False, bucket='', cache_images=False, image_weights=False, device='mps', multi_scale=False, single_cls=False, adam=False, sync_bn=False, local_rank=-1, workers=8, project='runs/train', entity=None, name='yolov7-car-detector', exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias='latest', freeze=[0], v5_metric=False, world_size=1, global_rank=-1, save_dir='runs/train/yolov7-car-detector4', total_batch_size=2)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_

# Run the testing 

In [5]:
!python test.py  \
    --device 0 \
    --batch-size 2 \ 
    --data rodent.yaml \
    --img 512  \
    --weights runs/train/yolov7-car-detector/weights/best.pt_xx \
    --name yolov7-rodent-detector

IndentationError: unexpected indent (4097609160.py, line 2)

## Export the model to ONNX

In [ ]:
!python export.py \
    --weights runs/train/yolov7-car-detector/weights/best.pt \
    --grid \
    --simplify \
    --img-size 512 512

# Add metadata for `Deepness` plugin to run the model smoothly

In [ ]:
import json
import onnx

model = onnx.load('runs/train/yolov7-car-detector3/weights/best.onnx')

class_names = {
    0: 'car',
}

m1 = model.metadata_props.add()
m1.key = 'model_type'
m1.value = json.dumps('Detector')

m2 = model.metadata_props.add()
m2.key = 'class_names'
m2.value = json.dumps(class_names)

m3 = model.metadata_props.add()
m3.key = 'resolution'
m3.value = json.dumps(10)

m4 = model.metadata_props.add()
m4.key = 'tiles_overlap'
m4.value = json.dumps(10)

m4 = model.metadata_props.add()
m4.key = 'det_conf'
m4.value = json.dumps(0.3)

m4 = model.metadata_props.add()
m4.key = 'det_iou_thresh'
m4.value = json.dumps(0.7)


FIANL_MODEL_FILE_PATH = os.path.abspath('runs/train/yolov7-car-detector2/weights/car_aerial_detection_yolo7_ITCVD_deepness.onnx')
onnx.save(model, FIANL_MODEL_FILE_PATH)

print(f'Your ONNX model with metadata is at: {FIANL_MODEL_FILE_PATH}')